In [125]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random

In [126]:
class DSO:
	def __init__(self, totalHours, marginalCost, unitSalePrice):
		self.totalHours = totalHours
		self.numberOfAggregators = 0
		self.marginalCost = marginalCost
		self.unitSalePrice = unitSalePrice
		self.aggregators = []
		self.maxUtility = 0

	def addAggregator(self, aggregator):
		self.aggregators.append(aggregator)
		self.numberOfAggregators += 1

	def getAggregators(self):
		return self.aggregators

	def Sfunction(self, aggregator, tIndex):
		return aggregator.maximumDemand[tIndex] * aggregator.maxPriceReference * (1 - np.exp(-aggregator.preferenceSatisfaction[tIndex] * (aggregator.bestLoadResponse[tIndex] / aggregator.nominalDemand[tIndex])))
	
	def CostFunction(self, aggregator, tIndex):
		return self.marginalCost[tIndex]

	def utilityFunction(self, m, theta, omega):
		sum = 0
		for i in range(self.numberOfAggregators):
			agg = self.aggregators[i]
			for j in range(self.totalHours):
				sum = sum + self.unitSalePrice[j] * agg.bestLoadResponse[j] - self.CostFunction[j] * agg.bestLoadResponse[j] + omega * self.Sfunction(agg, j)
		sum = sum - theta*m
		return sum

class Aggregator:
	def __init__(self, totalHours, nominalDemand, minimumDemand, maximumDemand, maxPriceReference, preferenceSatisfaction):
		self.totalHours = totalHours
		self.nominalDemand = nominalDemand
		self.minimumDemand = minimumDemand
		self.maximumDemand = maximumDemand
		self.maxPriceReference = maxPriceReference
		self.preferenceSatisfaction = preferenceSatisfaction
		self.actualLoad = 0
		self.currentUtility = 0
		self.bestLoadResponse = []

	def utilityFunction(self, DSO):
		sum = 0
		for i in range(DSO.totalHours):
			sum = sum + DSO.Sfunction(self, i) - DSO.unitSalePrice[i] * self.bestLoadResponse[i]
		return sum

	def optimalDR(self, DSO):
		for j in range(DSO.totalHours):
			self.bestLoadResponse.append(
			 (self.nominalDemand[j] / self.preferenceSatisfaction[j]) * (np.log((self.preferenceSatisfaction[j] * self.maxPriceReference) / DSO.unitSalePrice[j]))
			)


In [127]:
datasetPath = '../dataset/'

In [128]:
numOfLA = 3
# numOfHours = 25919
numOfHours = 200
LAs = []

In [129]:
# simulate a dataset for 3 houses for 1 year with random values for each column
# columns for each house: nominal demand, minimum demand, maximum demand, max price reference, preference satisfaction
# columns for DSO: marginal cost, unit sale price

for i in range(numOfLA):
	nominalDemand = np.random.randint(1, 100, numOfHours)
	minimumDemand = np.random.randint(1, 100, numOfHours)
	maximumDemand = np.random.randint(1, 100, numOfHours)
	maxPriceReference = np.random.randint(1, 100)
	preferenceSatisfaction = np.random.randint(1, 100, numOfHours)
	LAs.append(Aggregator(numOfHours, nominalDemand, minimumDemand, maximumDemand, maxPriceReference, preferenceSatisfaction))


In [130]:
print(len(LAs[0].nominalDemand))

200


In [131]:
# print all the values for each house
for i in range(numOfLA):
	print("LA", i)
	print("nominal demand: ", LAs[i].nominalDemand)
	print("minimum demand: ", LAs[i].minimumDemand)
	print("maximum demand: ", LAs[i].maximumDemand)
	print("max price reference: ", LAs[i].maxPriceReference)
	print("preference satisfaction: ", LAs[i].preferenceSatisfaction)

LA 0
nominal demand:  [45 84 18 58 14 92 23 32 88 97 85 26 10 27 79 42 89 84 77 14 39 34 11 16
 28 21 31 92 73 93 92 56  9 17 77 25 24 69 61 46 35 46 56 15 56 71 46 67
 63 56 67 20 76 35 63 95 22 22 36 27 46 60 90 67 69 84 69 57 51 29 10 48
 39 35  4  3  2 20 75 58 52 95 59 11 76 54 53 14 49 34 93 61 52  6 39 11
 29 44 90 97 86 55 87 34 56 59  8 28 29 55 62 31 41 77  8 25 77 58  3 61
 16 43 36 85 87 70 96 67 21 56 42 90 92 44 92 12 85 46 44 43 91 68 28 46
  3  5 76 26 96 41 95 81 86 80 68 15 29 94 66 56 23 41 95  2 54  5 92 48
 92  1 52 62 75 82 89 98 57 81 66 74 99 23 81 50 68 94 31 33 64  4 56 97
 90 75  9 19 66 17 82 79]
minimum demand:  [42 47 85 70 40  5 25 77 82 29 60 89 55 75  9 39 82 41 29 44 60 96 27 59
 18 47  7 32 26 19  2 74 65 66 61 73 53  4 66 44 28  9  7 40 46 14 40 95
 57  3 60 21  1 88 91 57 37 21 43 48 58 31 86 97 92 56 27 89 98 63 77 16
 38 43 19 14 43 81 98 22 60 94 94 18 70 30 30  4 17 95 42 69  3 79 18 91
 76 35  1 87 55 57 83 38 88 67 65  1 98 73 51 46 12  3 23 3

In [132]:
# create one DSO and input random values for marginal cost and unit sale price

marginalCost = np.random.randint(1, 100, numOfHours)
unitSalePrice = np.random.randint(1, 100, numOfHours)
dso = DSO(numOfHours, marginalCost, unitSalePrice)



In [133]:
# print all the values for the DSO
print("DSO")
print("marginal cost: ", dso.marginalCost)
print("unit sale price: ", dso.unitSalePrice)


DSO
marginal cost:  [20 98 18 15 67 56 21 37 15 41 24 14 65 57 30 39  5 14 74 76 78 20 53 71
 49 14  7 58  2 77 37 52 36 47 92 14  3 24 74 24  3 65 31 58 25 80 29 24
 39 98  8 94 33 58 79 15 50 67 80 40 72 11 21 36 68 93  1 83 13 88 26 48
  8  1 38 23 35 72 28 99  5 53 58 57 36 96 96  4 71  7 14 34 77 15 41 86
 47  9 71 17 96 67 26 99 54 90 34 66 40  6 29 85 46 20 90 29 10 34 87 36
 53 58 10 22 17 69 80 61 89 10  6 45 57 31 40 35 69 51 10 68 90 54 27 84
 11 15 79  4 66 69 25  6 67 45 65 94 69 78 81  5 79 60 63 65 37  1 69  7
 56 63 19 95 51 64 52  8 58 85 45 54 45 96 38 92 60 20  3  7 77 55 65 80
 81 51 47 39 87 50  6 91]
unit sale price:  [66 45 67 49  4 61 69 39 96  6 68 95 55 32 72 81  1 14 93 47 92 31 45 36
 34 57 76 22 39 48 94 94  2 94 68 91 83  6 92 23 84 60 80 58 14 37 42 51
 66 42 43 75 71 85  9 82 96 48 87 58 33 10  7  9 13  8 32 28  9 32 63 45
 62 47 92 24 53 11 96 95 98 74 78 85 53 45 24 96  5 68  6 12 10 28 76 36
 66 77 18 43 50 37 31 50 64 89  4 43 72  4 68 77 84 40 20 50

In [134]:
# DSO gives the price for each hour
for i in range(numOfLA):
	la = LAs[i]
	la.optimalDR(dso)

In [135]:
# print best load response for each house
for i in range(numOfLA):
	print("LA", i)
	print("best load response: ", LAs[i].bestLoadResponse)

LA 0
best load response:  [4.608652632217128, 4.68214615051328, 0.9667164479549745, 2.6953985034044075, 3.7332666488293578, 5.548673428422019, 3.087374059669382, 4.992694231417372, 9.080751796088743, 13.708245681042813, 3.1522542013317074, 1.380071149512606, 0.7205174521560063, 1.9360492013666333, 4.009001700494127, 2.074016166741286, 9.249046182693043, 15.34054960176086, 2.6307323833384353, 0.6037466280305038, 1.8249374884841103, 3.191191824906915, 1.6112980233798941, 0.8596919554313663, 1.8489265836925595, 0.85476437546004, -32.08179070274206, 5.030176488947556, 4.96202642905956, 10.35183028719042, 5.69952674737142, 5.9087121896788775, 2.028733692738184, -21.20678457651647, 3.97849504890718, 1.3317661726528454, 1.6678932477923278, 5.024670766048904, 5.282514967971496, 6.398593254144323, 1.458557040461227, 2.6506218526392744, 1.984849339010526, 1.4725618081956664, 5.038468420987259, 3.1936508003163584, 2.244695734692548, 12.088916990288652, 3.3605530205100047, 8.46054514227317, 10.402

In [136]:
# DSO takes back the load response from each aggregator
# calculate the utility for DSO with random m, theta and omega

m = np.random.randint(1, 100)
theta = np.random.randint(1, 100)
omega = np.random.randint(1, 100)

dso.maxUtility = dso.utilityFunction(m, theta, omega)
dso.maxUtility
    

-5742

In [139]:
# print utility for each house
for i in range(numOfLA):
	print("LA", i)
	ut = LAs[i].utilityFunction(dso)
	print("utility: ", ut)

LA 0
utility:  215416.35387919334
LA 1
utility:  841628.7587269727
LA 2
utility:  309759.141766019


In [138]:
# import gameanalysis
# import numpy as np

# # Define the DSO and aggregator objects
# totalHours = 24
# marginalCost = np.random.rand(totalHours)
# unitSalePrice = np.random.rand(totalHours)
# maxPriceReference = 1
# preferenceSatisfaction = np.random.rand(totalHours)
# actualLoad = np.random.rand(totalHours)
# nominalDemand = np.random.rand(totalHours)
# minimumDemand = np.random.rand(totalHours)
# maximumDemand = np.random.rand(totalHours)

# dso = gameanalysis.StackelbergGame(DSO.numberOfAggregators)
# for i in range(DSO.numberOfAggregators):
#     agg = DSO.aggregators[i]
#     u = np.zeros((totalHours, totalHours))
#     for j in range(totalHours):
#         u[j, :] = dso.devfn(DSO.utilityFunction, agg, j)
#     dso.add_player(u)

# # Calculate the equilibrium strategy for the DSO
# dso_eq = dso.solve()[0]

# # Calculate the equilibrium strategies for each aggregator
# for i in range(DSO.numberOfAggregators):
#     agg = DSO.aggregators[i]
#     agg_strategies = []
#     for j in range(totalHours):
#         s = dso_eq.strategies[agg.index][j, :]
#         agg_strategies.append(s)
#     agg_eq = gameanalysis.regret.mixture_regret(
#         agg_strategies, DSO.aggregators[i].utilityFunction)
